In [5]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Asumiendo que min_date_str y max_date_str están disponibles de la ejecución de la celda anterior
# Si no, necesitarías ejecutar la celda anterior nuevamente o establecerlos manualmente
# Nota: Como comentamos la celda anterior, necesitarás definir min_date_str y max_date_str manualmente
# Por ejemplo:
min_date_str = '2015-01-01T00:00:00.000' # Reemplaza con la fecha mínima real si la conoces
max_date_str = '2026-08-09T23:59:59.999' # Reemplaza con la fecha máxima real si la conoces


fecha_min = datetime.fromisoformat(min_date_str.replace('Z', '+00:00')) if 'Z' in min_date_str else datetime.fromisoformat(min_date_str)
fecha_max = datetime.fromisoformat(max_date_str.replace('Z', '+00:00')) if 'Z' in max_date_str else datetime.fromisoformat(max_date_str)

rangos_trimestrales = []
fecha_inicio_actual = fecha_min

while fecha_inicio_actual <= fecha_max:
    fin_de_trimestre = fecha_inicio_actual + relativedelta(months=3) - relativedelta(days=1)
    # Asegurar que la fecha de fin no exceda la fecha_max
    if fin_de_trimestre > fecha_max:
        fin_de_trimestre = fecha_max

    rangos_trimestrales.append((fecha_inicio_actual, fin_de_trimestre))

    fecha_inicio_actual = fecha_inicio_actual + relativedelta(months=3)

# Mostrar los rangos trimestrales generados
for inicio, fin in rangos_trimestrales:
    print(f"Inicio: {inicio.strftime('%Y-%m-%d')}, Fin: {fin.strftime('%Y-%m-%d')}")

Inicio: 2015-01-01, Fin: 2015-03-31
Inicio: 2015-04-01, Fin: 2015-06-30
Inicio: 2015-07-01, Fin: 2015-09-30
Inicio: 2015-10-01, Fin: 2015-12-31
Inicio: 2016-01-01, Fin: 2016-03-31
Inicio: 2016-04-01, Fin: 2016-06-30
Inicio: 2016-07-01, Fin: 2016-09-30
Inicio: 2016-10-01, Fin: 2016-12-31
Inicio: 2017-01-01, Fin: 2017-03-31
Inicio: 2017-04-01, Fin: 2017-06-30
Inicio: 2017-07-01, Fin: 2017-09-30
Inicio: 2017-10-01, Fin: 2017-12-31
Inicio: 2018-01-01, Fin: 2018-03-31
Inicio: 2018-04-01, Fin: 2018-06-30
Inicio: 2018-07-01, Fin: 2018-09-30
Inicio: 2018-10-01, Fin: 2018-12-31
Inicio: 2019-01-01, Fin: 2019-03-31
Inicio: 2019-04-01, Fin: 2019-06-30
Inicio: 2019-07-01, Fin: 2019-09-30
Inicio: 2019-10-01, Fin: 2019-12-31
Inicio: 2020-01-01, Fin: 2020-03-31
Inicio: 2020-04-01, Fin: 2020-06-30
Inicio: 2020-07-01, Fin: 2020-09-30
Inicio: 2020-10-01, Fin: 2020-12-31
Inicio: 2021-01-01, Fin: 2021-03-31
Inicio: 2021-04-01, Fin: 2021-06-30
Inicio: 2021-07-01, Fin: 2021-09-30
Inicio: 2021-10-01, Fin: 202

In [6]:
import requests
import pandas as pd
from dateutil.relativedelta import relativedelta
import time # Importar time para añadir retrasos
import os # Importar os para manipulación de rutas

# Asegúrate de que Google Drive esté montado antes de ejecutar esta celda
# from google.colab import drive
# drive.mount('/content/drive')


datos_trimestrales_completos = []
url = "https://www.datos.gov.co/resource/p6dx-8zbt.json" # URL corregida con el nuevo dataset ID
directorio_salida = '/content/drive/MyDrive/DatosAPI_Trimestral_Parquet/' # Directorio donde se guardarán los archivos en formato Parquet

# Crear el directorio si no existe
os.makedirs(directorio_salida, exist_ok=True)

# Asumiendo que rangos_trimestrales está disponible de la ejecución de la celda anterior

for fecha_inicio, fecha_fin in rangos_trimestrales:
    fecha_inicio_str = fecha_inicio.strftime('%Y-%m-%d')
    fecha_fin_str = fecha_fin.strftime('%Y-%m-%d')
    print(f"Obteniendo datos para el trimestre: {fecha_inicio_str} a {fecha_fin_str}")

    params = {
        "$where": f"fecha_de_publicacion_del >= '{fecha_inicio.strftime('%Y-%m-%dT%H:%M:%S')}' AND fecha_de_publicacion_del <= '{fecha_fin.strftime('%Y-%m-%dT%H:%M:%S')}'",
        "$limit": 50000 # Límite aumentado, pero aún es bueno manejar la paginación potencial si es necesario
    }

    res = requests.get(url, params=params)

    if res.status_code == 200:
        data = res.json()
        if data: # Procesar solo si hay datos
            df_trimestre = pd.DataFrame(data)
            # Generar nombre del archivo basado en el rango de fechas con extensión .parquet
            nombre_archivo = f"datos_trimestre_{fecha_inicio.year}_Q{(fecha_inicio.month-1)//3 + 1}_{fecha_inicio_str}_to_{fecha_fin_str}.parquet"
            ruta_archivo = os.path.join(directorio_salida, nombre_archivo)
            df_trimestre.to_parquet(ruta_archivo, index=False) # Cambiado a to_parquet
            print(f"Guardados {len(data)} registros en {ruta_archivo}")
        else:
            print(f"No se obtuvieron registros para este trimestre: {fecha_inicio_str} a {fecha_fin_str}.")
    else:
        print(f"Error al obtener datos para {fecha_inicio_str} a {fecha_fin_str}: {res.status_code}")
        print(res.text) # Imprimir respuesta de error para depuración

    time.sleep(1) # Añadir un pequeño retraso para evitar saturar la API

print("Descarga de datos para todos los trimestres finalizada.")

Obteniendo datos para el trimestre: 2015-01-01 a 2015-03-31
No se obtuvieron registros para este trimestre: 2015-01-01 a 2015-03-31.
Obteniendo datos para el trimestre: 2015-04-01 a 2015-06-30
Guardados 313 registros en /content/drive/MyDrive/DatosAPI_Trimestral_Parquet/datos_trimestre_2015_Q2_2015-04-01_to_2015-06-30.parquet
Obteniendo datos para el trimestre: 2015-07-01 a 2015-09-30
Guardados 2617 registros en /content/drive/MyDrive/DatosAPI_Trimestral_Parquet/datos_trimestre_2015_Q3_2015-07-01_to_2015-09-30.parquet
Obteniendo datos para el trimestre: 2015-10-01 a 2015-12-31
Guardados 342 registros en /content/drive/MyDrive/DatosAPI_Trimestral_Parquet/datos_trimestre_2015_Q4_2015-10-01_to_2015-12-31.parquet
Obteniendo datos para el trimestre: 2016-01-01 a 2016-03-31
Guardados 596 registros en /content/drive/MyDrive/DatosAPI_Trimestral_Parquet/datos_trimestre_2016_Q1_2016-01-01_to_2016-03-31.parquet
Obteniendo datos para el trimestre: 2016-04-01 a 2016-06-30
Guardados 3002 registros e

In [7]:
import pandas as pd
import os

output_dir = '/content/drive/MyDrive/DatosAPI_Trimestral_Parquet/' # Directorio donde se guardaron los archivos parquet

# Obtener una lista de todos los archivos parquet en el directorio
all_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith('.parquet')]

if all_files:
    print(f"Se encontraron {len(all_files)} archivos Parquet.")
    # Seleccionar algunos archivos para verificar (por ejemplo, el primero, el del medio y el último)
    files_to_verify = []
    if len(all_files) >= 1:
        files_to_verify.append(all_files[0])
    if len(all_files) >= 2:
        files_to_verify.append(all_files[len(all_files) // 2])
    if len(all_files) >= 3:
        files_to_verify.append(all_files[-1])
    # Eliminar duplicados si los hay
    files_to_verify = list(set(files_to_verify))

    # Configurar opciones de pandas para mostrar todas las columnas (opcional)
    pd.set_option('display.max_columns', None)

    for f in files_to_verify:
        print(f"\nVerificando archivo: {f}")
        try:
            df_verify = pd.read_parquet(f) # Cambiado a read_parquet
            print(f"Número de filas: {len(df_verify)}")
            print("Columnas:", df_verify.columns.tolist())
            display(df_verify.head())
        except Exception as e:
            print(f"Error al leer el archivo {f}: {e}")

    # Restablecer opciones de pandas (opcional)
    # pd.reset_option('display.max_columns')

else:
    print("No se encontraron archivos Parquet en el directorio especificado.")

Se encontraron 42 archivos Parquet.

Verificando archivo: /content/drive/MyDrive/DatosAPI_Trimestral_Parquet/datos_trimestre_2020_Q3_2020-07-01_to_2020-09-30.parquet
Número de filas: 50000
Columnas: ['entidad', 'nit_entidad', 'departamento_entidad', 'ciudad_entidad', 'ordenentidad', 'codigo_pci', 'id_del_proceso', 'referencia_del_proceso', 'ppi', 'id_del_portafolio', 'nombre_del_procedimiento', 'descripci_n_del_procedimiento', 'fase', 'fecha_de_publicacion_del', 'fecha_de_ultima_publicaci', 'fecha_de_publicacion_fase_3', 'precio_base', 'modalidad_de_contratacion', 'justificaci_n_modalidad_de', 'duracion', 'unidad_de_duracion', 'fecha_de_recepcion_de', 'fecha_de_apertura_de_respuesta', 'fecha_de_apertura_efectiva', 'ciudad_de_la_unidad_de', 'nombre_de_la_unidad_de', 'proveedores_invitados', 'proveedores_con_invitacion', 'visualizaciones_del', 'proveedores_que_manifestaron', 'respuestas_al_procedimiento', 'respuestas_externas', 'conteo_de_respuestas_a_ofertas', 'proveedores_unicos_con', 

,entidad,nit_entidad,departamento_entidad,ciudad_entidad,ordenentidad,codigo_pci,id_del_proceso,referencia_del_proceso,ppi,id_del_portafolio,nombre_del_procedimiento,descripci_n_del_procedimiento,fase,fecha_de_publicacion_del,fecha_de_ultima_publicaci,fecha_de_publicacion_fase_3,precio_base,modalidad_de_contratacion,justificaci_n_modalidad_de,duracion,unidad_de_duracion,fecha_de_recepcion_de,fecha_de_apertura_de_respuesta,fecha_de_apertura_efectiva,ciudad_de_la_unidad_de,nombre_de_la_unidad_de,proveedores_invitados,proveedores_con_invitacion,visualizaciones_del,proveedores_que_manifestaron,respuestas_al_procedimiento,respuestas_externas,conteo_de_respuestas_a_ofertas,proveedores_unicos_con,numero_de_lotes,estado_del_procedimiento,id_estado_del_procedimiento,adjudicado,id_adjudicacion,codigoproveedor,departamento_proveedor,ciudad_proveedor,fecha_adjudicacion,valor_total_adjudicacion,nombre_del_adjudicador,nombre_del_proveedor,nit_del_proveedor_adjudicado,codigo_principal_de_categoria,estado_de_apertura_del_proceso,tipo_de_contrato,subtipo_de_contrato,categorias_adicionales,urlproceso,codigo_entidad,estado_resumen,fecha_de_publicacion_fase_2,fecha_de_publicacion,fecha_de_publicacion_fase
0,SUBRED INTEGRADA DE SERVICIOS DE SALUD NORTE E...,900971006,Distrito Capital de Bogotá,No Definido,Territorial,Descentralizada,CO1.REQ.1359690,INVITACIÓN A COTIZAR No 082-2020 S.NTE,702729500,CO1.BDOS.1318397,MATERIAL DE OSTEOSINTESIS,SE REQUIERE CONTRATAR LOS INSUMOS PARA EL SUMI...,Presentación de oferta,2020-07-01T00:00:00.000,2020-07-01T00:00:00.000,2020-07-01T00:00:00.000,800000000,Contratación régimen especial (con ofertas),Regla aplicable,2,Mes(es),2020-07-10T00:00:00.000,2020-07-10T00:00:00.000,2020-07-10T00:00:00.000,Bogotá,DIRECCIÓN DE CONTRATACIÓN - COMPRAS,0,0,40,0,0,0,0,30,0,Seleccionado,70,Si,CO1.AWD.800423,703801464,Distrito Capital de Bogotá,No Definido,2020-08-11T00:00:00.000,15000000,ANDRES MAURICIO MOJICA ALAVA,SMITH & NEPHEW,900124455,V1.42321500,Cerrado,Suministros,No Definido,"V142321600, V142321700, V142321800, V142321900...",{'url': 'https://community.secop.gov.co/Public...,702729500,Adjudicado,None,None,None
1,ECOPETROL S.A.,899999068,Distrito Capital de Bogotá,Bogotá,Nacional,Centralizada,CO1.REQ.1359804,3032899,709072474,CO1.BDOS.1318503,SUMINISTRO DE TRANSMISORES E INTERRUPTORES,SUMINISTRO DE TRANSMISORES E INTERRUPTORES PAR...,Presentación de oferta,2020-07-01T00:00:00.000,2020-07-01T00:00:00.000,2020-07-01T00:00:00.000,904413023,Contratación régimen especial,Regla aplicable,71,día(s),None,None,None,Bogotá,Departamento de Compras,0,0,0,0,0,0,0,0,0,Publicado,50,No,No Adjudicado,No Definido,No Definido,No Definido,None,0,No Adjudicado,No Definido,No Definido,V1.41103309,Abierto,Suministros,No Definido,No definido,{'url': 'https://community.secop.gov.co/Public...,709072474,Presentación de oferta,None,None,None
2,GOBERNACIÓN DE BOYACÁ,891800498,Boyacá,Tunja,Territorial,Centralizada,CO1.REQ.1355513,CD-0728-2020,702727819,CO1.BDOS.1314613,PRESTACIÓN DE SERVICIOS,PRESTAR SERVICIOS DE TÉCNICO CON DESPLAZAMIENT...,Fase de ofertas,2020-07-01T00:00:00.000,2020-07-01T00:00:00.000,2020-07-01T00:00:00.000,15082540,Contratación directa,Servicios profesionales y apoyo a la gestión,179,día(s),None,None,None,Tunja,SECRETARIA CONTRATACION,0,0,0,0,0,0,0,0,0,Seleccionado,70,No,No Adjudicado,709656334,Boyacá,Tibaná,None,0,No Adjudicado,FELIPE INFANTE SUAREZ,1054803097,V1.80111620,Abierto,Prestación de servicios,No Definido,No definido,{'url': 'https://community.secop.gov.co/Public...,702727819,Adjudicado,None,None,None
3,MUNICIPIO DE SINCELEJO,800104062,Sucre,Sincelejo,Territorial,Descentralizada,CO1.REQ.1359060,CPS 688-2020,706481892,CO1.BDOS.1318058,BRINDAR APOYO AL AREA DE PRESUPUESTO,BRINDAR APOYO AL AREA DE PRESUPUESTO DE LA ALC...,Presentación de oferta,2020-07-01T00:00:00.000,2020-07-01T00:00:00.000,2020-07-01T00:00:00.000,17800000,Contratación directa,Servicios profesionales y apoyo a la gestión,6,Mes(es),None,None,None,No Aplic


Verificando archivo: /content/drive/MyDrive/DatosAPI_Trimestral_Parquet/datos_trimestre_2015_Q2_2015-04-01_to_2015-06-30.parquet
Número de filas: 313
Columnas: ['entidad', 'nit_entidad', 'departamento_entidad', 'ciudad_entidad', 'ordenentidad', 'codigo_pci', 'id_del_proceso', 'referencia_del_proceso', 'ppi', 'id_del_portafolio', 'nombre_del_procedimiento', 'descripci_n_del_procedimiento', 'fase', 'fecha_de_publicacion_del', 'fecha_de_ultima_publicaci', 'fecha_de_publicacion_fase_3', 'precio_base', 'modalidad_de_contratacion', 'justificaci_n_modalidad_de', 'duracion', 'unidad_de_duracion', 'ciudad_de_la_unidad_de', 'nombre_de_la_unidad_de', 'proveedores_invitados', 'proveedores_con_invitacion', 'visualizaciones_del', 'proveedores_que_manifestaron', 'respuestas_al_procedimiento', 'respuestas_externas', 'conteo_de_respuestas_a_ofertas', 'proveedores_unicos_con', 'numero_de_lotes', 'estado_del_procedimiento', 'id_estado_del_procedimiento', 'adjudicado', 'id_adjudicacion', 'codigoproveedor

,entidad,nit_entidad,departamento_entidad,ciudad_entidad,ordenentidad,codigo_pci,id_del_proceso,referencia_del_proceso,ppi,id_del_portafolio,nombre_del_procedimiento,descripci_n_del_procedimiento,fase,fecha_de_publicacion_del,fecha_de_ultima_publicaci,fecha_de_publicacion_fase_3,precio_base,modalidad_de_contratacion,justificaci_n_modalidad_de,duracion,unidad_de_duracion,ciudad_de_la_unidad_de,nombre_de_la_unidad_de,proveedores_invitados,proveedores_con_invitacion,visualizaciones_del,proveedores_que_manifestaron,respuestas_al_procedimiento,respuestas_externas,conteo_de_respuestas_a_ofertas,proveedores_unicos_con,numero_de_lotes,estado_del_procedimiento,id_estado_del_procedimiento,adjudicado,id_adjudicacion,codigoproveedor,departamento_proveedor,ciudad_proveedor,valor_total_adjudicacion,nombre_del_adjudicador,nombre_del_proveedor,nit_del_proveedor_adjudicado,codigo_principal_de_categoria,estado_de_apertura_del_proceso,tipo_de_contrato,subtipo_de_contrato,categorias_adicionales,urlproceso,codigo_entidad,estado_resumen,fecha_de_publicacion_fase_2,fecha_de_recepcion_de,fecha_de_apertura_de_respuesta,fecha_de_apertura_efectiva,fecha_adjudicacion
0,EMPRESA DE ACUEDUCTO Y ALCANTARILLADO DE BOGOT...,899999094,Distrito Capital de Bogotá,Bogotá,Territorial,Descentralizada,CO1.REQ.501,ISG-0334-2015,700064108,CO1.BDOS.501,PRESTACIÓN DE SERVICIOS,No definido,Presentación de oferta,2015-04-16T00:00:00.000,2015-04-16T00:00:00.000,2015-04-16T00:00:00.000,0,Contratación régimen especial,Regla aplicable,9,Mes(es),Bogotá,UNIDAD DE CONTRATACION DE COMPRAS,0,0,0,0,0,0,0,0,0,Evaluación,60,No,No Adjudicado,No Definido,No Definido,No Definido,0,No Adjudicado,No Definido,No Definido,V1.80111701,Abierto,Prestación de servicios,No Definido,No definido,{'url': 'https://community.secop.gov.co/Public...,700064108,Presentación de oferta,None,None,None,None,None
1,EMPRESA DE ACUEDUCTO Y ALCANTARILLADO DE BOGOT...,899999094,Distrito Capital de Bogotá,Bogotá,Territorial,Descentralizada,CO1.REQ.601,ICGH-0330-2015,700064108,CO1.BDOS.601,PRESTACIÓN DE SERVICIOS,APOYAR A LA DIRECCIÓN MEJORAMIENTO CALIDAD DE ...,Presentación de oferta,2015-04-20T00:00:00.000,2015-04-20T00:00:00.000,2015-04-20T00:00:00.000,0,Contratación régimen especial,Regla aplicable,10,Mes(es),Bogotá,UNIDAD DE CONTRATACION DE COMPRAS,0,0,0,0,0,0,0,0,0,Publicado,50,No,No Adjudicado,No Definido,No Definido,No Definido,0,No Adjudicado,No Definido,No Definido,V1.83101500,Abierto,Prestación de servicios,No Definido,No definido,{'url': 'https://community.secop.gov.co/Public...,700064108,Presentación de oferta,None,None,None,None,None
2,EMPRESA DE ACUEDUCTO Y ALCANTARILLADO DE BOGOT...,899999094,Distrito Capital de Bogotá,Bogotá,Territorial,Descentralizada,CO1.REQ.605,IJ-0354-2015,700064108,CO1.BDOS.606,PRESTACIÓN DE SERVICIOS,PRESTACIÓN DE SERVICIOS PROFESIONALES PARA EJE...,Presentación de oferta,2015-04-20T00:00:00.000,2015-04-20T00:00:00.000,2015-04-20T00:00:00.000,0,Contratación régimen especial,Regla aplicable,10,Mes(es),Bogotá,UNIDAD DE CONTRATACION DE COMPRAS,0,0,0,0,0,0,0,0,0,Publicado,50,No,No Adjudicado,No Definido,No Definido,No Definido,0,No Adjudicado,No Definido,No Definido,V1.83101500,Abierto,Prestación de servicios,No Definido,No definido,{'url': 'https://community.secop.gov.co/Public...,700064108,Presentación de oferta,None,None,None,None,None
3,EMPRESA DE ACUEDUCTO Y ALCANTARILLADO DE BOGOT...,899999094,Distrito Capital de Bogotá,Bogotá,Territorial,Descentralizada,CO1.REQ.602,IF-0332-2015,700064108,CO1.BDOS.602,PRESTACIÓN DE SERVICIOS,APOYAR A LA DIRECCIÓN DE JURISDICCIÓN COACTIVA...,Presentación de oferta,2015-04-20T00:00:00.000,2015-04-20T00:00:00.000,2015-04-20T00:00:00.000,0,Contratación régimen especial,Regla aplicable,11,Mes(es),Bogotá,UNIDAD DE CONTRATACION DE COMPRAS,0,0,0,0,0,0,0,0,0,Publicado,50,No,No Adjudicado,No Definido,No Definido,No Definido,0,No Adjudicado,No Definido,No Definido,V1.83101500,Abierto,Prestación de servicios,No Definido,No definido,{'url': 'https://comm


Verificando archivo: /content/drive/MyDrive/DatosAPI_Trimestral_Parquet/datos_trimestre_2025_Q3_2025-07-01_to_2025-09-30.parquet
Número de filas: 50000
Columnas: ['entidad', 'nit_entidad', 'departamento_entidad', 'ciudad_entidad', 'ordenentidad', 'codigo_pci', 'id_del_proceso', 'referencia_del_proceso', 'ppi', 'id_del_portafolio', 'nombre_del_procedimiento', 'descripci_n_del_procedimiento', 'fase', 'fecha_de_publicacion_del', 'fecha_de_ultima_publicaci', 'fecha_de_publicacion_fase_3', 'precio_base', 'modalidad_de_contratacion', 'justificaci_n_modalidad_de', 'duracion', 'unidad_de_duracion', 'ciudad_de_la_unidad_de', 'nombre_de_la_unidad_de', 'proveedores_invitados', 'proveedores_con_invitacion', 'visualizaciones_del', 'proveedores_que_manifestaron', 'respuestas_al_procedimiento', 'respuestas_externas', 'conteo_de_respuestas_a_ofertas', 'proveedores_unicos_con', 'numero_de_lotes', 'estado_del_procedimiento', 'id_estado_del_procedimiento', 'adjudicado', 'id_adjudicacion', 'codigoproveed

,entidad,nit_entidad,departamento_entidad,ciudad_entidad,ordenentidad,codigo_pci,id_del_proceso,referencia_del_proceso,ppi,id_del_portafolio,nombre_del_procedimiento,descripci_n_del_procedimiento,fase,fecha_de_publicacion_del,fecha_de_ultima_publicaci,fecha_de_publicacion_fase_3,precio_base,modalidad_de_contratacion,justificaci_n_modalidad_de,duracion,unidad_de_duracion,ciudad_de_la_unidad_de,nombre_de_la_unidad_de,proveedores_invitados,proveedores_con_invitacion,visualizaciones_del,proveedores_que_manifestaron,respuestas_al_procedimiento,respuestas_externas,conteo_de_respuestas_a_ofertas,proveedores_unicos_con,numero_de_lotes,estado_del_procedimiento,id_estado_del_procedimiento,adjudicado,id_adjudicacion,codigoproveedor,departamento_proveedor,ciudad_proveedor,valor_total_adjudicacion,nombre_del_adjudicador,nombre_del_proveedor,nit_del_proveedor_adjudicado,codigo_principal_de_categoria,estado_de_apertura_del_proceso,tipo_de_contrato,subtipo_de_contrato,categorias_adicionales,urlproceso,codigo_entidad,estado_resumen,fecha_de_recepcion_de,fecha_de_apertura_de_respuesta,fecha_de_apertura_efectiva,fecha_adjudicacion,fecha_de_publicacion_fase_2,fecha_de_publicacion
0,HOSPITAL DEPARTAMENTAL MARIA INMACULADA ESE,891180098,Caquetá,No Definido,Nacional,Centralizada,CO1.REQ.8493326,01895-2025,704428127,CO1.BDOS.8343114,DANNA LEANDRA ZAPATA FIERRO,PRESTAR SUS SERVICIOS PERSONALES EN FORMA INDE...,Presentación de oferta,2025-07-01T00:00:00.000,2025-07-01T00:00:00.000,2025-07-01T00:00:00.000,7590000,Contratación régimen especial,Decreto 092 de 2017,90,día(s),Florencia,OFICINA DE CONTRATACIÓN,0,0,0,0,0,0,0,0,0,Publicado,50,No,No Adjudicado,No Definido,No Definido,No Definido,0,No Adjudicado,No Definido,No Definido,V1.85101601,Abierto,Decreto 092 de 2017,No Definido,No definido,{'url': 'https://community.secop.gov.co/Public...,704428127,Presentación de oferta,None,None,None,None,None,None
1,UNIVERSIDAD DE NARIÑO,800118954,Nariño,Pasto,Territorial,Centralizada,CO1.REQ.8489239,CSU-252131,700358047,CO1.BDOS.8339011,CONTRATO DE PRESTACION DE SERVICIOS,Contratar servicios profesionales de un Médico...,Presentación de oferta,2025-07-01T00:00:00.000,2025-07-01T00:00:00.000,2025-07-01T00:00:00.000,11880000,Contratación régimen especial,Decreto 092 de 2017,5,Mes(es),Pasto,CENTRO DE ESTUDIOS EN SALUD DE LA UNIVERSIDAD ...,0,0,0,0,0,0,0,0,0,Publicado,50,No,No Adjudicado,No Definido,No Definido,No Definido,0,No Adjudicado,No Definido,No Definido,V1.80111600,Abierto,Decreto 092 de 2017,No Definido,No definido,{'url': 'https://community.secop.gov.co/Public...,700358047,Presentación de oferta,None,None,None,None,None,None
2,UNIVERSIDAD NACIONAL DE COLOMBIA,899999063,Distrito Capital de Bogotá,No Definido,Nacional,Centralizada,CO1.REQ.8491074,UNAL-OPS-2025-2012-164,700698038,CO1.BDOS.8340759,PRESTAR SERVICIOS TÉCNICOS EN LAS ACTIVIDADES ...,PRESTAR SERVICIOS TÉCNICOS EN LAS ACTIVIDADES ...,Presentación de oferta,2025-07-01T00:00:00.000,2025-07-01T00:00:00.000,2025-07-01T00:00:00.000,12299584,Contratación régimen especial,Decreto 092 de 2017,92,día(s),Bogotá,Bogotá Fac. Artes,0,0,0,0,0,0,0,0,0,Publicado,50,No,No Adjudicado,No Definido,No Definido,No Definido,0,No Adjudicado,No Definido,No Definido,V1.80111600,Abierto,Decreto 092 de 2017,No Definido,No definido,{'url': 'https://community.secop.gov.co/Public...,700698038,Presentación de oferta,None,None,None,None,None,None
3,MUNICIPIO DE ALTAMIRA,891180118,Huila,Altamira,Territorial,Descentralizada,CO1.REQ.8490407,MACPS-065-2025,704897024,CO1.BDOS.8339693,PRESTAR LOS SERVICIOS DE APOYO A LA GESTIÓN PA...,PRESTAR LOS SERVICIOS DE APOYO A LA GESTIÓN PA...,Presentación de oferta,2025-07-01T00:00:00.000,2025-07-01T00:00:00.000,2025-07-01T00:00:00.000,16380000,Contratación directa,Servicios profesionales y apoyo a la gestión,6,Mes(es),Altamira,MUNICIPIO DE ALTAMIRA,0,0,0,0,0,0,0,0,0,Seleccionado,70,No,No Adjudicado,718480775,Huila,Isnos,0,No Adjudicado,YOIN ALEJANDRO MUÑOZ NAVIA,1084259471,V1.80111620,Abierto,Prest